# Finansielle tabeller
- Lager alle finansielle tabeller:
    1. Finansielle grunnlagsdata
    2. Finansielle nøkkeltall i kr per innbygger
    3. Finansielle nøkkeltall i prosent av langsiktig gjeld
    4. Finansielle nøkkeltall i prosent av brutto driftsinntekter
    5. Utvalgte nøkkeltall
- For regnskapsomfang
    1. Kasse
    2. Konsoliderte
    3. Konsern
- For region
    1. Kommune
    2. Fylkeskommune

## Henter nødvendige pakker

In [ ]:
import pandas as pd
import numpy as np

## Angi årgang som skal produseres

In [ ]:
%run ./../aargang.ipynb

## Henter data

In [ ]:
bev_konsern = pd.read_parquet('/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/'+aar+'/bev_konsern.parquet')
bev_kako = pd.read_parquet('/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/'+aar+'/bev_kasse_konsolidert.parquet')

In [ ]:
bal_konsern= pd.read_parquet('/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/'+aar+'/finansielt_grunnlag_ink_rente.parquet')
bal_kako = pd.read_parquet('/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/'+aar+'/bal_kasse_konsolidert.parquet')

## Fjerner bydel
- Lager ikke finansielle tabeller for bydel
**Er bydel fjernet for kasse? Om ikke kan de fjernes tidligere i løpet.**

In [ ]:
bydel = bev_konsern[(bev_konsern['region'].str.contains('0301'))
                    & (bev_konsern['region'].str[4:]!='')]
bydelsliste = bydel.region.unique()

In [ ]:
bev_konsern_ub = bev_konsern[(~bev_konsern['region'].isin(bydelsliste)) &
                            (bev_konsern['region'] != 'EAB')]

In [ ]:
bal_konsern_ub = bal_konsern[(~bal_konsern['region'].isin(bydelsliste)) &
                            (bal_konsern['region'] != 'EAB')]

# Finansielle grunnlagsdata
## Kommunekonsern

In [ ]:
filter_bev = pd.read_csv("/ssb/bruker/mtk/Testdata/inndata/Filter Finansielle grunnlagsdata.csv", sep=';')

- Filtrerer ut de variablene som skal være med fra bevilgningsregnskapet

In [ ]:
keys = list(filter_bev.columns.values)
i1 = bev_konsern_ub.set_index(keys).index
i2 = filter_bev.set_index(keys).index
fingr_bev= bev_konsern_ub[i1.isin(i2)]
fingr_bev = fingr_bev.drop(columns=['funksjon', 'kontoklasse','regnskapsomfang']).reset_index(drop=True)
fingr_bev.rename(columns={'art':'art_kap'}, inplace = True)

- Filtrerer ut de verdiene som skal være med fra balanseregnskapet

In [ ]:
filter_bal = ["5900", "KG14", "KG15", "KG28", "KG38", "56", "KG21", "KG25", "KG32", "KG18", "KG31", "KG40", "40", "KG39", "41", "42", "43"]
fingr_bal = bal_konsern_ub[(bal_konsern_ub['kapittel'].isin(filter_bal))]
fingr_bal.rename(columns={'kapittel':'art_kap'}, inplace = True)

- Samler bevilgning og balanse

In [ ]:
fingr_konsern = pd.concat([fingr_bal, fingr_bev])

In [ ]:
#Fylkeskommune. Denne stemmer ikke siden det ikke er riktig filter
fingn_komkonsern = fingr_konsern.loc[(fingr_konsern['region'].str[2:]!='00')&
                                     (~fingr_konsern['region'].str.contains('EAFK'))]

### Henter kommuner som ikke har levert skjema 23
- Skjema hentes i egen notebook. Her skapes innrapporteringssiden. 

In [ ]:
%run ./Auto_grunnlag/mangler_skjema_23.ipynb

### Filtrerer ut KG39 og KG40 for kommuner som ikke har levert skjema 23

In [ ]:
skjema_23 = skjema_23.drop(columns={'kommunenavn'})
skjema_23['KG39'] = 'KG39'
skjema_23['KG40'] = 'KG40'
skjema_23_miss = skjema_23.melt('region', value_name='art_kap').drop(columns={'variable'})

In [ ]:
skjema_23_miss['marker']= 1

In [ ]:
joined = pd.merge(fingn_komkonsern, skjema_23_miss, on=['region', 'art_kap'], how='left')
finansielle_grunnlagsdata_kommunekonsern = joined[pd.isna(joined['marker'])][fingn_komkonsern.columns]